In [16]:
# https://github.com/van12809957/vf-upou-dcs

In [17]:
import os
import pandas as pd
import csv
import json

In [18]:
!python --version

Python 3.11.0


In [19]:
# listing all the files I have
os.listdir()

['.git',
 '.gitconfig',
 '.gitignore',
 'components.puml',
 'data_transpo.csv',
 'README.md',
 'Testing_Codebasics_Graph_Implementation.ipynb']

# Class

In [41]:
# adjusting the code from codebasics to be able to take more dimensions

# decided to declare a Node class which will help in expanding the dimensions available per node in the graph
# what happens is graph_dict now contains a dictionary. Dictionary within a dictionary. 

class Node:
    def __init__(self):
        """ 
            Generates a Node that represents each location in a map. 
        
            destination_dict = dictionary that contains the destination for each source and accompanying details. Expandable. 


            Template:

            {
                destination_1: {
                    traveltime: _____ (in minutes),
                    mode: _____,
                },
                destination_2: {
                    traveltime: _____ (in minutes),
                    mode: _____,
                }
            }

            Example:

            {
                'ABC Hospital': {
                    traveltime: 30,
                    mode: car
                },
                'XYZ School': {
                    traveltime: 5,
                    mode: tricycle
                }
            }

        """
        
        self.destination_dict = {}

    def addDestination(self, data_dict):
        new_destination = data_dict['destination']

        # you may expand the details of each node through the following code
        # not every column from the data_dict parameter is automatically stored in graph_dict
        cols_to_store = ['mode', 'traveltime', 'destination_type']

        node_details_dict = {}
        
        for elem in cols_to_store: 
            node_details_dict[elem] = data_dict[elem]

        self.destination_dict[new_destination] = node_details_dict


class Graph:
    def __init__(self, data, isHeader=True):
        """
        Generates the graph using provided data. 

        data = List of lists. Minimum expected data per list is: [source, destination, mode, traveltime]
        isHeader = does the data have column headers
        """

        # the original self.graph_dict() is utilized to determine paths.
        # however this limits the capability of the graph dict to consider path weights and other data
        # there are numerous approaches on how we can modify the code such as creating a new class for the node
        # thus this modification maintains the graph_dict logic
            # see Node class above for easier visualization of what graph_dict will contain
            # code below can also be run 

        # checks if there is a column header for the dataframe (2D array)
        if isHeader:
            col_headers = data[0]
            self.edges = data[1:]
        else:
            # if no column header, then will default to the one in the doc string above
            col_headers = ['source', 'destination', 'mode', 'traveltime']
            self.edges = data

        self.graph_dict = {}

        for row in self.edges:
            temp_dict = {}
            source = row[col_headers.index('source')]

            # to avoid hardcoding and overhauling code at every expansion
            # throw all the data in dict and then let the addDestination handle all the parsing
            # and which elements to include or not
            for elem in col_headers:
                temp_dict[elem] = row[col_headers.index(elem)]

            if source in self.graph_dict:
                self.graph_dict[source].addDestination(
                    temp_dict
                )
            else:
                myNode = Node()
                myNode.addDestination(temp_dict)
                self.graph_dict[source] = myNode

    def print_graph(self):
        """Prints the contents of the graph"""
        for key in self.graph_dict:
            print('SOURCE:', key)
            print(json.dumps(self.graph_dict[key].destination_dict, indent=4))
            print('='*25)
        print('*'*50)


    def print_edges(self):
        """Prints the edges of the graph"""
        print('Edges:')
        print([k for k in self.graph_dict])
        print('*'*50)

    
    # UNFINISHED
    # def view_nearest_facility(self, facilities='all'):
    #     if facilities=='all':
    #         for elem in self.graph_dict:
    #             # print only the facilities that are not labeled as house
    #     else:
    #         for facility in facilities:
    #             # print the facilities that are provided as a list
        

if __name__ == '__main__':

    myGraph = Graph(data_transpo)

# Code Demo

In [42]:
## we could utilize pandas but will stick with base python for now to practice
# data_transpo = pd.read_csv('data_transpo.csv')
# data_transpo

# we could modify the class to accept arrays instead of tuples directly from the loading of data

with open("data_transpo.csv", "r", encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    data_transpo = list(reader)

# preview sample data that will be used for the demo
data_transpo

[['source', 'destination', 'mode', 'traveltime', 'destination_type'],
 ['A', 'ABC Elementary School', 'car', '15', 'School'],
 ['A', 'Smith Hospital', 'car', '7', 'Hospital'],
 ['B', 'ABC Elementary School', 'car', '10', 'School'],
 ['B', 'Smith Hospital', 'walk', '20', 'Hospital'],
 ['C', 'ABC Elementary School', 'car', '7', 'School'],
 ['C', 'Barangay KNL Tricycle Terminal', 'walk', '5', 'Tricycle Terminal'],
 ['T', 'ABC Elementary School', 'tricycle', '3', 'School']]

In [43]:
# Create the graph
myGraph = Graph(data_transpo)

In [46]:
# Hi Jan, we can change this name. Just selected this generic name for now

# Created a loop that shows the options users may perform with the graph
# generated from the loaded data
def showMenu():
    print('.\n'*8)
    print('Welcome to the Urban Transport Planning Simulator!')
    print('*'*22, 'Menu:', '*'*22)
    print("""
    1 - View the Graph generated using the data
    2 - View all direct destinations and relevant details from a specific location
    3 - View the edges
    4 - View the nearest facilities in a specific location
    # - Exit
        """)
    print('*'*51)
    userMenuSelection = input('Select an action to perform by typing the corresponding number and pressing Enter (e.g., 2):').strip()
    return userMenuSelection

In [48]:
# After loading the data, users are presented the following choices which act like 
# a menu of the available features in our notebook
# Note that I haven't yet done a lot of exception, edge cases handling

# if we want to expand the operation per choice we can create separate functions, to keep it cleaner
while(True):

    userMenuSelection = showMenu()
    
    match userMenuSelection:
        case '#':
            print('Thank you for using the Urban Transport Planning Simulator!')
            break
        case '1':
            print('GRAPH GENERATED CONTAINS THE FOLLOWING:')
            myGraph.print_graph()
            continue
        case '2':     
            userInput = input('Enter selected location:')
            userInput = userInput.strip()
            print('Direct destinations and relevant details for location:', userInput)
            for elem in myGraph.graph_dict[userInput].destination_dict:
                print('Destination:', elem)
                print(myGraph.graph_dict[userInput].destination_dict[elem])
            continue
        case '3':
            print('GRAPH CONTAINS THE FOLLOWING EDGES:')
            myGraph.print_edges()
            continue
        case _:
            continue


.
.
.
.
.
.
.
.

Welcome to the Urban Transport Planning Simulator!
********************** Menu: **********************

    1 - View the Graph generated using the data
    2 - View all direct destinations and relevant details from a specific location
    3 - View the edges
    4 - View the nearest facilities in a specific location
    # - Exit
        
***************************************************
GRAPH GENERATED CONTAINS THE FOLLOWING:
SOURCE: A
{
    "ABC Elementary School": {
        "mode": "car",
        "traveltime": "15",
        "destination_type": "School"
    },
    "Smith Hospital": {
        "mode": "car",
        "traveltime": "7",
        "destination_type": "Hospital"
    }
}
SOURCE: B
{
    "ABC Elementary School": {
        "mode": "car",
        "traveltime": "10",
        "destination_type": "School"
    },
    "Smith Hospital": {
        "mode": "walk",
        "traveltime": "20",
        "destination_type": "Hospital"
    }
}
SOURCE: C
{
    "ABC Elementary 

In [ ]:
# # from codebasics
# # this is not built for expansion...

# class Graph:
#     def __init__(self, edges):
#         """
#         edges = List containing tuples with two elements... need to adjust to become open for multiple dimensions

        
#         """
#         self.edges = edges
#         self.graph_dict = {}
#         for start, end in edges:
#             if start in self.graph_dict:
#                 self.graph_dict[start].append(end)
#             else:
#                 self.graph_dict[start] = [end]
#         print("Graph Dict:", self.graph_dict)

#     def get_paths(self, start, end, path=[]):
#         path = path + [start]

#         if start == end:
#             return [path]

#         if start not in self.graph_dict:
#             return []

#         paths = []
#         for node in self.graph_dict[start]:
#             if node not in path:
#                 new_paths = self.get_paths(node, end, path)
#                 for p in new_paths:
#                     paths.append(p)

#         return paths

#     def get_shortest_path(self, start, end, path=[]):
#         path = path + [start]
#         # Multipliers should also be stored and parsed and we need to get sum?

#         if start == end:
#             return path

#         if start not in self.graph_dict:
#             return None

#         shortest_path = None
#         for node in self.graph_dict[start]:
#             if node not in path:
#                 sp = self.get_shortest_path(node, end, path)
#                 if sp:
#                     if shortest_path is None or len(sp) < len(shortest_path):
#                         shortest_path = sp
#                         # in this line of code we would likely need to keep a track of the type of transportation and the multiplier (weighted edge)
#                         # instead of start end only, node should have a multiplier value as well.
#         return shortest_path

# if __name__ == '__main__':

#     routes = [
#         ("Mumbai","Pune"),
#         ("Mumbai", "Surat"),
#         ("Surat", "Bangaluru"),
#         ("Pune","Hyderabad"),
#         ("Pune","Mysuru"),
#         ("Hyderabad","Bangaluru"),
#         ("Hyderabad", "Chennai"),
#         ("Mysuru", "Bangaluru"),
#         ("Chennai", "Bangaluru")
#     ]

#     routes = [
#         ("Mumbai", "Paris"),
#         ("Mumbai", "Dubai"),
#         ("Paris", "Dubai"),
#         ("Paris", "New York"),
#         ("Dubai", "New York"),
#         ("New York", "Toronto"),
#     ]

#     route_graph = Graph(routes)

#     start = "Mumbai"
#     end = "New York"

#     print(f"All paths between: {start} and {end}: ",route_graph.get_paths(start,end))
#     print(f"Shortest path between {start} and {end}: ", route_graph.get_shortest_path(start,end))

#     start = "Dubai"
#     end = "New York"

#     print(f"All paths between: {start} and {end}: ",route_graph.get_paths(start,end))
#     print(f"Shortest path between {start} and {end}: ", route_graph.get_shortest_path(start,end))